In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 
os.environ["WORLD_SIZE"] = "1"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "/home/quant/.cache/modelscope/hub/qwen/Qwen1___5-0___5B-Chat", 
    torch_dtype=torch.float16, 
        # cache_dir=cache_dir, 
        low_cpu_mem_usage=True, 
        device_map="auto"
)
print(model.device)
tokenizer = AutoTokenizer.from_pretrained("/home/quant/.cache/modelscope/hub/qwen/Qwen1___5-0___5B-Chat")

# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(device)

# generated_ids = model.generate(
#     model_inputs.input_ids,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


cuda:0


In [3]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt")

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

/home/quant/.conda/envs/AWQ/lib/python3.10/site-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [13]:
print(response)

A Large Language Model (LLM) is a type of machine learning algorithm that is designed to process and generate text in a large dataset, which can be much larger than the training data used by traditional machine learning algorithms. LLMs are trained using a combination of supervised and unsupervised learning techniques, such as neural networks and clustering algorithms, to learn patterns and relationships in large amounts of text data.
One key advantage of LLMs is their ability to generate high-quality, coherent and relevant text based on the input they receive. This makes them well-suited for tasks such as chatbots, sentiment analysis, language translation, and content generation.
However, there are also some challenges associated with developing and deploying large language models, such as the need to handle large amounts of data, the need to optimize the performance of the model, and the need to maintain a large and efficient computing infrastructure. As with any artificial intellige

In [2]:
import torch 
tensor = torch.randint(0, 100, size=1000)  # 生成一个0到99之间的1000个随机整数的数组

# 使用numpy.unique计算每个元素的出现次数
unique_elements, counts = np.unique(tensor, return_counts=True)

# 计算出现次数后5%和前5%的元素的阈值
sorted_counts_indices = np.argsort(counts)  # 获取按出现次数排序后的索引
n = len(counts)  # 唯一元素的数量
threshold_5_percent = int(n * 0.05)  # 计算5%的元素数量

# 获取出现次数后5%和前5%的元素
elements_last_5_percent = unique_elements[sorted_counts_indices[-threshold_5_percent:]]
elements_first_5_percent = unique_elements[sorted_counts_indices[:threshold_5_percent]]

elements_last_5_percent, elements_first_5_percent

TypeError: randint() received an invalid combination of arguments - got (int, int, size=int), but expected one of:
 * (int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int low, int high, tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [33]:
import torch

def low_rank_block(blocks, rank=1):
    # Perform batched SVD
    u, s, v = torch.linalg.svd(blocks,full_matrices=False)
    # Keep only the `rank` largest singular values and corresponding vectors
    U_k = u[:, :, :rank] # 对 U 裁剪到前 k 列
    Vh_k = v[:, :rank, :] # 对 Vh 裁剪到前 k 行
    S_k = s[:, :rank]
    # Use batched matrix multiplication to reconstruct the low-rank blocks
    S_k_diag = torch.zeros(S_k.shape[0], rank, rank).cuda()
    S_k_diag= torch.diag_embed(S_k)
    low_rank = torch.bmm(torch.bmm(U_k, S_k_diag), Vh_k)
    return low_rank

def process_tensor(tensor, rank=1, block_size=4):
    # Determine the size of the tensor
    rows, cols = tensor.size()
    # Reshape tensor to a batch of blocks
    blocks = tensor.unfold(0, block_size, block_size).unfold(1, block_size, block_size)
    blocks = blocks.contiguous().view(-1, block_size, block_size)
    # Perform low-rank approximation on each block in parallel
    low_rank_blocks = low_rank_block(blocks, rank=rank)
    # low_rank_blocks = blocks
    # Reshape the output back to the original shape
    output = low_rank_blocks.view(rows // block_size, cols // block_size, block_size, block_size)
    output = output.permute(0, 2, 1, 3).reshape(rows, cols)
    return output

# Assume tensor is on GPU and in the desired precision (e.g. half-precision)
tensor = torch.randn(8, 8, device='cuda')
tensor2 = tensor.clone()
# Process the tensor
output = process_tensor(tensor)
print(output-tensor)

tensor([[ 1.2952e-01, -2.6574e-01, -2.7647e-01, -3.2114e-01, -3.2628e-04,
          1.9058e-01,  1.0286e+00,  1.1706e+00],
        [-3.6464e-01,  2.1215e-01,  2.1455e-02, -6.4072e-02,  1.4150e-01,
          1.0430e+00,  4.9595e-01,  1.3037e-01],
        [-1.9009e-01,  7.8974e-02, -2.7699e-02,  1.2472e-01, -3.4673e-02,
         -4.4958e-02, -3.8283e-01, -3.0454e-01],
        [ 4.4659e-01, -7.7999e-02,  2.3043e-01,  4.0573e-01,  3.5865e-02,
          3.8204e-01,  3.4960e-01,  2.9479e-01],
        [-1.1705e+00, -8.0482e-01, -3.0533e-01, -4.8692e-01, -1.0049e-01,
         -8.1237e-01,  5.9837e-01, -2.5156e-01],
        [-4.6196e-01, -1.6335e-01, -2.7533e-01,  1.0167e-01, -4.1416e-01,
          4.8535e-01, -3.9036e-02,  1.4397e+00],
        [-5.6295e-01,  6.5870e-01, -2.9037e-01,  9.7767e-01, -3.4209e-01,
          2.4369e-01, -9.3074e-02,  1.0253e+00],
        [ 7.5815e-01,  3.8997e-01,  1.0179e+00, -5.2737e-01,  1.8256e-01,
         -3.6175e-01,  3.2991e-02, -7.6089e-01]], device='cuda:0'

In [34]:
def low_rank_decomposition(weight, rank=1):
    """
    :param          weight: The matrix to decompose, of shape (H, W)
    :param    reduced_rank: the final rank
    :return:
    """

    """parameter_ratio = rank * (H + W) / (H * W)"""
    """rank_ratio = """
    matrix_dimension = len(weight.size())
    assert matrix_dimension == 2, "Only Support 2D matrix"
    H, W = weight.size()

    # Use SVD to decompose a matrix, default full_matrices is False to save parameters
    U, S, Vh = torch.linalg.svd(weight, full_matrices=False)

    L = U @ (torch.sqrt(torch.diag(S)[:, 0:rank]))
    R = torch.sqrt(torch.diag(S)[0:rank, :]) @ Vh

    # print(f"W: ({H},{W}) | Rank: {rank} | U:{U.shape} | S:{S.shape} | Vh:{Vh.shape}")
    # print(f"Reduced Rank: {reduced_rank} | Num Parameters: {(H + W) * reduced_rank}")
    print(f"L: {L.shape} | R: {R.shape}")

    return L@R
def process_tensor_simple(tensor, rank=1):
    # Determine the size of the tensor
    rows, cols = tensor.size()
    # Initialize a tensor for the output
    output = torch.empty_like(tensor)
    
    # Iterate over the tensor in 4x4 blocks
    for i in range(0, rows, 4):
        for j in range(0, cols, 4):
            # Perform low-rank approximation on each block
            block = tensor[i:i+4, j:j+4]
            output[i:i+4, j:j+4] = low_rank_decomposition(block, rank=rank)
    return output

output = process_tensor_simple(tensor2)
print(output-tensor2)

L: torch.Size([4, 1]) | R: torch.Size([1, 4])
L: torch.Size([4, 1]) | R: torch.Size([1, 4])
L: torch.Size([4, 1]) | R: torch.Size([1, 4])
L: torch.Size([4, 1]) | R: torch.Size([1, 4])
tensor([[ 1.2952e-01, -2.6574e-01, -2.7647e-01, -3.2114e-01, -3.2634e-04,
          1.9058e-01,  1.0286e+00,  1.1706e+00],
        [-3.6464e-01,  2.1215e-01,  2.1456e-02, -6.4072e-02,  1.4150e-01,
          1.0430e+00,  4.9595e-01,  1.3037e-01],
        [-1.9009e-01,  7.8974e-02, -2.7699e-02,  1.2472e-01, -3.4673e-02,
         -4.4958e-02, -3.8283e-01, -3.0454e-01],
        [ 4.4659e-01, -7.7999e-02,  2.3043e-01,  4.0573e-01,  3.5865e-02,
          3.8204e-01,  3.4960e-01,  2.9479e-01],
        [-1.1705e+00, -8.0482e-01, -3.0533e-01, -4.8692e-01, -1.0049e-01,
         -8.1237e-01,  5.9837e-01, -2.5156e-01],
        [-4.6196e-01, -1.6335e-01, -2.7533e-01,  1.0167e-01, -4.1416e-01,
          4.8535e-01, -3.9036e-02,  1.4397e+00],
        [-5.6295e-01,  6.5870e-01, -2.9037e-01,  9.7767e-01, -3.4209e-01,
    

In [1]:
import torch
x = torch.empty(5, 3).fill_(2)
t = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], dtype=torch.float)
index = torch.tensor([0, 4, 2, 0])
x.index_reduce_(0, index, t, 'mean')
x = torch.empty(5, 3).fill_(2)
x.index_reduce_(0, index, t, 'mean', include_self=False)

/tmp/ipykernel_536353/671234142.py:5: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1138.)
  x.index_reduce_(0, index, t, 'prod')


tensor([[10., 22., 36.],
        [ 2.,  2.,  2.],
        [ 7.,  8.,  9.],
        [ 2.,  2.,  2.],
        [ 4.,  5.,  6.]])